In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

In [202]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [170]:
t=pd.concat([train,test])

In [175]:
cols='Crop_Type Soil_Type Pesticide_Use_Category Season'.split() 
for col in cols:
    t=pd.get_dummies(t,columns=[col])

In [177]:
train=t.iloc[:len(train)]
test=t.iloc[len(train):]

In [203]:
from sklearn.impute import SimpleImputer

In [219]:
imp=SimpleImputer(missing_values=np.nan,strategy='constant')
X['Number_Weeks_Used']=imp.fit_transform(X[['Number_Weeks_Used']])

In [220]:
t['Number_Weeks_Used']=imp.fit_transform(t[['Number_Weeks_Used']])

In [218]:
t=test.drop(['ID'],axis=1)
X=train.drop(['Crop_Damage','ID'],axis=1)
y=train['Crop_Damage']

In [20]:
from sklearn.model_selection import train_test_split,GridSearchCV,StratifiedKFold

In [22]:
from lightgbm import LGBMClassifier

In [157]:
from xgboost import XGBClassifier

In [120]:
from sklearn.metrics import accuracy_score

In [161]:
t=test.drop(['ID'],axis=1)

In [221]:
accuracy=[]
models=[]
fold=StratifiedKFold(n_splits=3,shuffle=True,random_state=101)

In [222]:
for train_idx,test_idx in fold.split(X,y):
    X_train,X_test=X.iloc[train_idx],X.iloc[test_idx]
    y_train,y_test=y[train_idx],y[test_idx]
    lgb=LGBMClassifier(max_depth=6,n_estimators=120)
    lgb.fit(X_train,y_train)
    pred=lgb.predict(X_test)
    accuracy.append(accuracy_score(y_test,pred))
    models.append(lgb)

In [223]:
accuracy

[0.8464888588791357, 0.84695634558898, 0.8472264424862419]

In [224]:
pr=[]
for model in models:
    p=model.predict_proba(t)
    pr.append(p)

In [225]:
pr1=np.array(pr)

In [226]:
pr2=sum(pr1)/3

In [227]:
p=np.argmax(pr2,axis=1)

In [228]:
sub=pd.DataFrame(columns='ID Crop_Damage'.split())

In [229]:
sub['ID']=test['ID']

In [230]:
sub['Crop_Damage']=p

In [231]:
sub

,ID,Crop_Damage
0,F00000002,1
1,F00000007,1
2,F00000011,1
3,F00000013,1
4,F00000014,1
...,...,...
59305,F00155937,0
59306,F00155940,0
59307,F00155941,0
59308,F00155943,0


In [232]:
sub.to_csv('Mysol15.csv',index=False)